### Day 4 Exercise 1 Secure Design Review

In this group activity, you will take on the role of a security reviewer assessing a basic web application before deployment. The goal is to apply your knowledge of secure software design by identifying weaknesses in the system's architecture, assumptions, and implementation. Below you will find design notes, a test plan, and source code for a minimal Flask-based web service called SimpleReg, which performs basic user registration and message display.

While SimpleReg appears functional, it contains intentional security gaps that mirror common real-world oversights. As a team, conduct a structured Security Design Review, evaluating the application's functionality, data flows, and trust boundaries. This includes reviewing how user data is handled, identifying missing controls, and recommending mitigations using secure design principles. Be prepared to report findings and recommendations.

This is not just about finding bugs—it's about spotting architectural decisions that could be improved to reduce risk before code reaches production.

Use this simplified review checklist for your review (use the hardcopy if provided so you can include notes and comments).

---

## Security Design Review Checklist
### 1. Understand the System Context
- Are all application features (e.g., registration, welcome display) documented?
- Are trust boundaries (e.g., user input over HTTP) identified?
- Are the technologies and libraries (Flask, Python) clearly stated?

### 2. Authentication & Authorization
- Is there any authentication implemented for registration or viewing user data?
- Is access to `/welcome/<username>` restricted appropriately?
- Could one user impersonate or access another user’s data?

### 3. Input Validation & Output Handling
- Is user input validated or sanitized before storage?
- Is output encoded to prevent XSS in the welcome message?
- Can malicious input be used to manipulate the application's behavior?

### 4. Error Handling & Logging
- Are errors returned in a safe and user-friendly way?
- Could error messages leak system details (e.g., debug mode)?
- Is logging implemented or planned for future monitoring?

### 5. Secure Communications
- Is the app configured to run with HTTPS in production?

### 6. Dependency & Environment Management
- Are third-party libraries (e.g., Flask) kept up to date?
- Are there plans to manage secrets or configuration securely?

### 7. Review of Design Notes and Test Plan
- Do the design notes acknowledge key security assumptions and limitations?
- Does the test plan include basic security scenarios (e.g., XSS, input tampering)?
- Are there any missing test cases that would improve coverage (e.g., long input, malformed JSON)?


---

# **SimpleReg**
# Application Design Notes

## Purpose
A simple Flask web service for user registration and welcome message display.

## Functional Requirements
- Users submit username and email via a POST request to `/register`.
- The system stores this data in-memory (no database).
- Anyone can visit `/welcome/<username>` to see that user's welcome message.
- Output is returned as raw HTML.

## Technologies
- Python 3.11
- Flask 2.x

---

# Test Plan

## Manual Tests

### 1. Registration
- Submit valid JSON with username and email.
- Try submitting missing fields.
- Try registering the same username twice.

### 2. Welcome Page
- Access welcome page for a registered user.
- Access welcome page for an unregistered user.

### 3. Security Tests (Threat Simulation)
- Try XSS: Register with username = `<script>alert(1)</script>`
- Try injection in email field: `admin@example.com<script>`
- Manually test access to another user's info (no auth)

---

# Source Code

```
from flask import Flask, request, jsonify

app = Flask(__name__)
users = {}  # In-memory user store

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get("username")
    email = data.get("email")
    users[username] = email
    return jsonify({
      "message": f"User {username} registered with email {email}."})

@app.route('/welcome/<username>', methods=['GET'])
def welcome(username):
    email = users.get(username)
    if not email:
        return jsonify({"error": "User not found"}), 404

    return f"<h1>Welcome, {username}!</h1><p>Your email: {email}</p>"

if __name__ == '__main__':
    app.run(debug=True)
```


# Solution

# Security Design Review: SimpleReg

## 1. Understand the System Context

- The design notes clearly describe application functionality: user registration and welcome message display.
- Trust boundaries (e.g., input via HTTP requests) are implied but not explicitly documented.
- Technologies used (Python 3.11, Flask 2.x) are listed.

**Finding**: The context is generally described, but trust boundaries and data flows are not explicitly defined.
**Recommendation**: Add a simple diagram or narrative outlining data flows, external actors, and trust boundaries.

---

## 2. Authentication & Authorization

- No authentication is implemented; anyone can register and view user data.
- The `/welcome/<username>` route allows anyone to access information about any registered user.
- This enables impersonation or unauthorized access by guessing usernames.

**Finding**: There is no access control or identity verification.
**Recommendation**: Implement authentication (e.g., using Flask-Login or tokens) and restrict access to user-specific data.

---

## 3. Input Validation & Output Handling

- User input is accepted without any validation or sanitation.
- Output is rendered directly as HTML with user-supplied content.
- XSS is possible, as shown in the test plan's `<script>` injection example.

**Finding**: High risk of XSS and potential injection attacks.
**Recommendation**:
- Validate input types and formats (e.g., use `email-validator`).
- Sanitize or escape output using Flask's `render_template` or Jinja2’s autoescaping.

---

## 4. Error Handling & Logging

- Error messages are returned in JSON format (e.g., user not found), which is appropriate.
- Application runs in `debug=True` mode, exposing stack traces and internal info.
- Logging is not implemented.

**Finding**: Debug mode leaks sensitive info and there’s no event logging.
**Recommendation**:
- Set `debug=False` in production.
- Implement logging using the `logging` module to track important events.

---

## 5. Secure Communications

- The app does not enforce HTTPS.
- Flask’s built-in server is not secure for production.

**Finding**: Communication is assumed to occur over insecure channels.
**Recommendation**:
- Use HTTPS behind a reverse proxy (e.g., Nginx).
- Document HTTPS as a deployment requirement.

---

## 6. Dependency & Environment Management

- Dependencies are not pinned or scanned.
- No mention of `requirements.txt` or security scanning.

**Finding**: No formal dependency management process in place.
**Recommendation**:
- Add a `requirements.txt` with pinned versions.
- Use tools like `pip-audit` or `trivy` to detect vulnerable packages.

---

## 7. Review of Design Notes and Test Plan

- The design notes acknowledge intentional lack of security features.
- The test plan includes basic coverage (e.g., XSS), but edge cases are missing.

**Finding**: Test coverage is reasonable but could be expanded.
**Recommendation**:
- Add tests for malformed JSON, long strings, empty fields, and invalid HTTP methods.

## Revised / Improved SimpleReg Application Source Code

This revised version addresses the security issues identified in the above design review.

## Summary of Improvements

### 1. Input Validation
- Required fields: `username`, `email`, and `password` must be present and non-empty.
- Email addresses are validated using the `email-validator` library to prevent malformed input.
- Inputs are stripped of leading/trailing whitespace.

### 2. Authentication & Session Management
- A simple login mechanism is added using secure password hashing (`werkzeug.security`).
- Session-based authentication restricts access to the `/welcome` route.
- A logout route is included to clear the session.

### 3. Output Encoding
- The `/welcome` page uses `render_template_string` with Jinja2 autoescaping to prevent XSS vulnerabilities.

### 4. Logging
- Secure logging is implemented using the `logging` module.
- Events such as registration and login are recorded in a log file (`security_log.txt`) with timestamps.

### 5. Debug Mode
- The application disables `debug=True` to avoid revealing stack traces and server internals in production.

### 6. Miscellaneous Security Practices
- A strong, random secret key is used for session management via `os.urandom(24)`.
- `SESSION_COOKIE_HTTPONLY` is enabled to reduce the risk of client-side script access to the session cookie.

```
from flask import Flask, request, jsonify, render_template_string, session
from email_validator import validate_email, EmailNotValidError
import logging
from werkzeug.security import generate_password_hash, check_password_hash
import os

# Initialize Flask app
app = Flask(__name__)

# Set a strong secret key for session management
app.secret_key = os.urandom(24)

# Prevent JavaScript access to session cookie (basic defense against XSS)
app.config['SESSION_COOKIE_HTTPONLY'] = True

# Configure secure logging for registration and login events
logging.basicConfig(
    filename="security_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# In-memory user store; each user has an email and hashed password
users = {}

@app.route('/register', methods=['POST'])
def register():
    """Handle user registration with input validation and secure password storage."""
    data = request.get_json()

    # Strip whitespace and extract input fields
    username = data.get("username", "").strip()
    email = data.get("email", "").strip()
    password = data.get("password", "").strip()

    # Check for missing fields
    if not username or not email or not password:
        return jsonify({"error": "Username, email, and password are required."}), 400

    # Prevent duplicate registration
    if username in users:
        return jsonify({"error": "Username already registered."}), 400

    # Validate email format using external library
    try:
        validate_email(email)
    except EmailNotValidError:
        return jsonify({"error": "Invalid email address."}), 400

    # Store user data with hashed password (never store plaintext passwords)
    users[username] = {
        "email": email,
        "password_hash": generate_password_hash(password)
    }

    # Log the registration event
    logging.info(f"User '{username}' registered.")
    return jsonify({"message": f"User '{username}' registered successfully."}), 201

@app.route('/login', methods=['POST'])
def login():
    """Authenticate a user and start a secure session."""
    data = request.get_json()
    username = data.get("username", "").strip()
    password = data.get("password", "").strip()

    user = users.get(username)

    # Verify password using secure hash comparison
    if user and check_password_hash(user["password_hash"], password):
        session['username'] = username  # Store username in session
        logging.info(f"User '{username}' logged in.")
        return jsonify({"message": "Login successful."})

    # Do not reveal whether username or password was incorrect
    return jsonify({"error": "Invalid username or password."}), 401

@app.route('/welcome', methods=['GET'])
def welcome():
    """Display a personalized welcome page for authenticated users."""
    username = session.get('username')

    # Ensure the user is logged in and exists
    if not username or username not in users:
        return jsonify({"error": "Unauthorized access."}), 403

    user = users[username]

    # Render personalized content using Jinja2 autoescaping
    safe_html = render_template_string(
        "<h1>Welcome, {{ username }}</h1><p>Your email: {{ email }}</p>",
        username=username,
        email=user["email"]
    )
    return safe_html

@app.route('/logout', methods=['POST'])
def logout():
    """Log out the user by clearing their session."""
    session.pop('username', None)
    return jsonify({"message": "Logged out."})

if __name__ == '__main__':
    # Run the application in production-safe mode (no debug info)
    app.run(debug=False)
```